In [1]:
!pip install minsearch


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import json
from minsearch import AppendableIndex

In [3]:
from chat_assistant_1 import IPythonChatInterface, Tools, ChatAssistant

In [4]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [7]:
from openai import OpenAI
client = OpenAI()

In [8]:
# def search(query):
#     boost = {'question': 3.0, 'section': 0.5}
    
#     results = index.search(
#         query=query,
#         filter_dict={'course': 'data-engineering-zoomcamp'},
#         boost_dict=boost,
#         num_results=5,
#         output_ids=True
#     )

#     return results

# # search_tool = {
# #     "type": "function",
# #     "name": "search",
# #     "description": "Search the FAQ database",
# #     "parameters": {
# #         "type": "object",
# #         "properties": {
# #             "query": {
# #                 "type": "string",
# #                 "description": "Search query text to look up in the course FAQ."
# #             }
# #         },
# #         "required": ["query"],
# #         "additionalProperties": False
# #     }
# # }


# def add_entry(question, answer):
#     """
#     Add a new entry to the FAQ database.

#     Args:
#         question (str): The question to be added to the FAQ database.
#         answer (str): The corresponding answer to the question.
#     """
#     doc = {
#         'question': question,
#         'text': answer,
#         'section': 'user added',
#         'course': 'data-engineering-zoomcamp'
#     }
#     index.append(doc)


# # add_entry_tool = {
# #     "type": "function",
# #     "name": "add_entry",
# #     "description": "Add an entry to the FAQ database",
# #     "parameters": {
# #         "type": "object",
# #         "properties": {
# #             "question": {
# #                 "type": "string",
# #                 "description": "The question to be added to the FAQ database",
# #             },
# #             "answer": {
# #                 "type": "string",
# #                 "description": "The answer to the question",
# #             }
# #         },
# #         "required": ["question", "answer"],
# #         "additionalProperties": False
# #     }
# # }

In [9]:
# results = search('I just discovered the course. Can I still join now?')
# print(results[0]['text'])

In [10]:
#tools = [search_tool, add_entry_tool]

In [11]:
# response = client.responses.create(
#     model="gpt-4o-mini",
#     input="hello"
# )
# print(response.output_text)


In [12]:
# developer_prompt = """
# You're a course teaching assistant.
# You're given a question from a course student and your task is to answer it

# Before making any function calls, explain you reasoning behind calling the funtion.

# When searching in our FAQ, perform multiple search queries with differently phrased.

# In the end also ask a related question to make it more engaging !
# """.strip()

# #question = "what are the things I need to install and learn to do well in module 1?"


# chat_messages = [
#     {"role": "developer", "content": developer_prompt},
# ]

# response = client.responses.create(
#     model='gpt-4o-mini',
#     input=chat_messages, 
#     tools=tools
# )

In [13]:
# response.output[0].type

In [14]:
# response.output

In [15]:
#response.output[0].name

In [16]:
# def make_call(entry):
#     args = json.loads(entry.arguments)
#     f_name = entry.name
#     f = globals()[f_name]  
#     call_id = entry.call_id
#     results = f(**args)
#     output_json = json.dumps(results)
    
#     call_output = {
#         "type": "function_call_output",
#         "call_id": entry.call_id,
#         "output": output_json,
#     }
#     return call_output

In [17]:
interface = IPythonChatInterface()

In [18]:
# generate_description(search)

In [19]:
tools =  Tools()
# tools.add_tool(search)
# tools.add_tool(add_entry)
tools.get_tools()

[]

In [20]:
from typing import Any, Dict, List

class CourseFAQTools:

    def __init__(self, index):
        self.index = index

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the FAQ database for a given query.
    
        Args:
            query (str): The search query text to look up in the course FAQ.
    
        Returns:
            List[Dict[str, Any]]: A list of search results, each represented as a dictionary.
        """
        boost = {'question': 3.0, 'section': 0.5}
        
        results = self.index.search(
            query=query,
            filter_dict={'course': 'data-engineering-zoomcamp'},
            boost_dict=boost,
            num_results=5,
            output_ids=True
        )
    
        return results   
    
    def add_entry(self, question: str, answer: str) -> None:
        """
        Add a new entry to the FAQ database.
    
        Args:
            question (str): The question to be added to the FAQ database.
            answer (str): The corresponding answer to the question.
    
        Returns:
            None
        """
        doc = {
            'question': question,
            'text': answer,
            'section': 'user added',
            'course': 'data-engineering-zoomcamp'
        }
        self.index.append(doc)
    


In [21]:
faq_tools = CourseFAQTools(index)

In [22]:
#faq_tools.search('docker')

In [23]:
# from typing import List, Dict, Any

# def search(query: str) -> List[Dict[str, Any]]:
#     """
#     Search the FAQ database for a given query.

#     Args:
#         query (str): The search query text to look up in the course FAQ.

#     Returns:
#         List[Dict[str, Any]]: A list of search results, each represented as a dictionary.
#     """
#     boost = {'question': 3.0, 'section': 0.5}
    
#     results = index.search(
#         query=query,
#         filter_dict={'course': 'data-engineering-zoomcamp'},
#         boost_dict=boost,
#         num_results=5,
#         output_ids=True
#     )

#     return results


# def add_entry(question: str, answer: str) -> None:
#     """
#     Add a new entry to the FAQ database.

#     Args:
#         question (str): The question to be added to the FAQ database.
#         answer (str): The corresponding answer to the question.

#     Returns:
#         None
#     """
#     doc = {
#         'question': question,
#         'text': answer,
#         'section': 'user added',
#         'course': 'data-engineering-zoomcamp'
#     }
#     index.append(doc)


In [24]:
tools.get_tools()

[]

In [25]:
# tools.add_tool(faq_tools.search)
# tools.add_tool(faq_tools.add_entry)

In [26]:
tools.add_tools(faq_tools)

In [27]:
tools.get_tools()

[{'type': 'function',
  'name': 'add_entry',
  'description': 'Add a new entry to the FAQ database.\n\nArgs:\n    question (str): The question to be added to the FAQ database.\n    answer (str): The corresponding answer to the question.\n\nReturns:\n    None',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'question parameter'},
    'answer': {'type': 'string', 'description': 'answer parameter'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database for a given query.\n\nArgs:\n    query (str): The search query text to look up in the course FAQ.\n\nReturns:\n    List[Dict[str, Any]]: A list of search results, each represented as a dictionary.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'query parameter'}},
   'required': ['query'],
   'additionalProperties': False}}]

In [28]:
print(tools)

In [29]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Before making any function calls, explain your reasoning why you want to perform something.

When searching in our FAQ, perform multiple search queries with diffierently phrased questions.

At the end, as the user a question to make it more engaging.

Also add the new entry to the FAQ database.
""".strip()

interface = IPythonChatInterface()

chat = ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    interface=interface,
    llm_client=client
)

In [30]:
chat.run()

User: How do I do well in Module 1 ?


User: stop


Chat Ended
